In [219]:
import pandas as pd
from bs4 import BeautifulSoup
import unicodedata
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords as wn
from nltk.corpus import wordnet
from string import maketrans

import sklearn.datasets as datasets
from time import sleep

In [220]:
Question_set = pd.read_csv('./questions.csv',sep=';')
all_docs= [q for q in Question_set['Question']]
all_labels = [l for l in Question_set['Categorie']]

In [221]:
from nltk import download

def nltkDownload():
    """ Vous n'aurez surement pas les stopwords, il vous faudra les télécharger.
    """
    download()

def fromVectoBow(all_docs, vec, normalize=True):
    bow = vec.fit_transform(all_docs)
    bow = bow.tocsr() # permet de print
    if normalize:
        bow = normalizeBow(bow)
    return bow


from nltk.corpus import stopwords

def makeNltkStopWords(languages=['french', 'english', 'german', 'spanish']):
    stop_words = []
    for l in languages:
        for w in stopwords.words(l):
            stop_words.append(w.encode('utf-8')) #w.decode('utf-8') buggait... avec certains caractères
    return stop_words

In [222]:

import sklearn.feature_extraction.text as txt
    
def fromAllDocsToBow(all_docs, strip_accents=u'ascii', lowercase=True, \
                     preprocessor=None, stop_words=None, token_pattern=u"[\\w']+\\w\\b", \
                     analyzer=u'word', max_df=1.0, max_features=20000, vocabulary=None, \
                     binary=False, ngram_range=(1, 1), min_df=1, \
                     normalize=True):
    """ Depuis un liste de documents, génère une matrice sparse contenant les occurences des mots.
        A chaque mot est associé un identifiant grace à une table de hashage.
    """
    vec_param = txt.CountVectorizer(all_docs, strip_accents=strip_accents, lowercase=lowercase, preprocessor=preprocessor, \
                            stop_words=stop_words, token_pattern=token_pattern, analyzer=analyzer, max_df=max_df, \
                            max_features=max_features, vocabulary=vocabulary, binary=binary, ngram_range=ngram_range, \
                            min_df=min_df)
    bow = fromVectoBow(all_docs, vec_param, normalize)
    return bow, vec_param

def fromVectoBow(all_docs, vec, normalize=True):
    bow = vec.fit_transform(all_docs)
    bow = bow.tocsr() # permet de print
    if normalize:
        bow = normalizeBow(bow)
    return bow

def normalizeBow(bow):
    """ TFIDF : La somme de toutes les occurences des mots devient égale à 1
    """
    transformer = txt.TfidfTransformer(use_idf=False, smooth_idf=False)
    bow_norm = transformer.fit_transform(bow)
    return bow_norm   

In [223]:
import numpy as np
import sklearn.naive_bayes as nb
from sklearn import svm
from sklearn import linear_model as lin
from sklearn import cross_validation
from sklearn import tree


def crossValidation(clf, bow, all_labels, cv=5):
    X = bow
    y = all_labels
    scores = cross_validation.cross_val_score(clf, X, y, cv=5)
    np_scores = np.array(scores)
    mean = np_scores.mean()
    std = np_scores.std()
    return scores, mean, std 

def fit(clf, bow, all_labels):
    """ Indispensable pour obtenir les clf.coef_ utile à la descrimination des mots """
    X = bow
    y = all_labels
    clf.fit(X, y)
    return clf

def predict(clf, docs):
    return clf.predict(docs)

In [224]:
# Paramétrage
languages = ['french']
stop_words = makeNltkStopWords(languages) # si erreur executez nltkDownload()
analyzer = u'word' # {‘word’, ‘char’, ‘char_wb’}
ngram_range = (1, 1) # unigrammes
lowercase = True
token_pattern = u"[\\w']+\\w\\b" # 
max_df = 1.0 #default
min_df = 5. * 1./len(all_docs) # on enleve les mots qui apparaissent moins de 5 fois
max_features = 20000 # nombre de mots au total dans notre matrice sparse
binary = False # presence coding or counting
strip_accents = u'ascii' #  {‘ascii’, ‘unicode’, None}
preprocessor=None
vocabulary=None

# Vectorisation
bow, vec = fromAllDocsToBow(all_docs, strip_accents=strip_accents, lowercase=lowercase, preprocessor=preprocessor, \
                            stop_words=stop_words, token_pattern=token_pattern, analyzer=analyzer, max_df=max_df, \
                            max_features=max_features, vocabulary=vocabulary, binary=binary, ngram_range=ngram_range, \
                            min_df=min_df)

#print "Mots vectorisés du second document :"
#print bow

### On crée des modèles

In [225]:
### Test avec la normalisation

In [226]:
# Normalisation
bow = normalizeBow(bow)

### On implémente nos modèles

In [227]:
dt = tree.DecisionTreeClassifier()
# Modèles
clf = svm.LinearSVC() # SVM
clf_nb = nb.MultinomialNB() # Naive Bayes
clf_rl = lin.LogisticRegression() # regression logistique

models =[{'name': 'decision Tree','model' :clf},{'name' : 'naive bayes' , 'model' : clf_nb},{'name' : 'logistic regression','model' : clf_rl}]
# Cross-Validation
for model in models:
    print model['name']
    scores, mean, std  = crossValidation(model['model'], bow, all_labels, cv=2)
    print "Scores obtenus avec crossValidation :", scores
    print "Moyenne :", mean
    print "Ecart type :", std

decision Tree
Scores obtenus avec crossValidation : [ 0.88888889  0.77777778  0.88888889  0.75        1.        ]
Moyenne : 0.861111111111
Ecart type : 0.0895806416478
naive bayes
Scores obtenus avec crossValidation : [ 0.77777778  0.77777778  0.77777778  0.625       0.83333333]
Moyenne : 0.758333333333
Ecart type : 0.0700528900718
logistic regression
Scores obtenus avec crossValidation : [ 0.77777778  0.66666667  0.88888889  0.625       0.83333333]
Moyenne : 0.758333333333
Ecart type : 0.0992253949972


### Test d'un nouveau bot avec de nouvelles données

In [228]:
# Motifs; délais; contrat-de-travail ; 
#all_docs = getAllDocs(docs)

#comment = """Pour quels raisons résilier mon contrat de bail?"""
#comment2 = """Quand est-ce que je récupèrerais mon dépot de garantie?"""
comment = "Quand mon propriétaire me rendra t-il mon dépot de garantie?"

#comment_bad = """attempts"""

#comment_good = """enjoyable"""

all_docs.append(comment)
#all_docs.append(comment2)

len_docs = len(all_docs)

bow, vec = fromAllDocsToBow(all_docs, strip_accents=strip_accents, lowercase=lowercase, preprocessor=preprocessor, \
                            stop_words=stop_words, token_pattern=token_pattern, analyzer=analyzer, max_df=max_df, \
                            max_features=max_features, vocabulary=vocabulary, binary=binary, ngram_range=ngram_range, \
                            min_df=min_df)

clf = fit(clf, bow[:len_docs-1], all_labels)

#print 'Longueur des documents '+ len_docs
#print 'Longueur de '

pred = predict(clf, bow[len_docs-1])
#print "Classe du commentaire : ", pred[0]

prediction = pred[0]

if prediction == 0 :
    print "Il s'agit d'une question sur les motifs\n"
    sleep(3)
    print "Le propriétaire peut résilier un bail locatif :\n"
    sleep(3)
    print "s'il veut reprendre le logement pour y habiter ou loger un proche, s'il veut le vendre, ou pour les motifs sérieux suivants :\n"
    sleep(3)
    print "(non-paiement du loyer, non-souscription à une assurance de risques locatifs, troubles du voisinage).\n"
    sleep(3)
    print "S'il résilie le contrat pour une autre raison.\n"
    print "Je vous conseille les voies de recours suivantes : https://www.service-public.fr/particuliers/vosdroits/F31301."
elif prediction  == 1:
    print "Il s'agit d'une question sur les délais\n"
    sleep(2)
    print "Pour les contrats de bail d'une location nue ou meublée, la loi prévoit que le dépôt de garantie doit être restitué sous deux mois à compter de la restitution des clefs."
    sleep(2)
    print "Sous un mois si l'état des lieux de sortie correspond à celui établi à l'entrée"
    sleep(2)
        

Il s'agit d'une question sur les délais

Pour les contrats de bail d'une location nue ou meublée, la loi prévoit que le dépôt de garantie doit être restitué sous deux mois à compter de la restitution des clefs.
Sous un mois si l'état des lieux de sortie correspond à celui établi à l'entrée


Pour les contrats de bail d'une location nue ou meublée, la loi prévoit que le dépôt de garantie doit être restitué sous deux mois à compter de la restitution des clefs, sous un mois si l'état des lieux de sortie correspond à celui établi à l'entrée. En cas de difficultés dans la restitution du dépôt de garantie sous le délai indiqué, je vous conseille de lire l'article suivant : https://www.service-public.fr/particuliers/vosdroits/F31301.

### Get new document

In [229]:
#all_docs

['Quand doit on rendre le d\xc3\xa9p\xc3\xb4t de garantie',
 'Quand me restituera-t-on le d\xc3\xa9p\xc3\xb4t de garantie',
 'd\xc3\xa9p\xc3\xb4t de garantie propri\xc3\xa9taire restitution d\xc3\xa9lai',
 'quel d\xc3\xa9lai pour me rendre la caution',
 'combien de temps pour r\xc3\xa9cup\xc3\xa9rer ma caution',
 'combien de temps pour r\xc3\xa9cup\xc3\xa9rer un d\xc3\xa9p\xc3\xb4t de garantie',
 'Mon propri\xc3\xa9taire doit il me rendre mon d\xc3\xa9p\xc3\xb4t de garantie',
 'Mon propri\xc3\xa9taire nie devoir me rendre mon d\xc3\xa9p\xc3\xb4t de garantie',
 'Mon propri\xc3\xa9taire ne veut pas me rembourser ma caution depuis deux mois',
 'Mon locataire veut son d\xc3\xa9p\xc3\xb4t de garantie rapidement',
 'litige sur les d\xc3\xa9lais pour rendre la caution',
 'r\xc3\xa9duire les d\xc3\xa9lais de pr\xc3\xa9avis',
 'Quand restituer le d\xc3\xa9p\xc3\xb4t de garantie',
 'd\xc3\xa9lai l\xc3\xa9gal pour rembourser la caution',
 'Comment r\xc3\xa9silier mon contrat de location',
 'Quels